In [1]:
import os 
import json
import pandas as pd
from tqdm import tqdm

from data import  prepare_data_for_symbol, query_gpt4, create_dataset
from prompt import get_all_prompts
from data_infererence_fetch import get_curday, fetch_all_data, get_all_prompts_online
from indices import *

In [6]:
# START_DATE = "2022-12-31"
# END_DATE = "2023-05-31"
start_date = "2023-01-31"
end_date = "2023-12-31"
index_name = "crypto"
min_past_weeks = 1
max_past_weeks = 4
train_ratio = 0.8

data_dir = f"./data/{index_name}_{start_date}_{end_date}"
os.makedirs(data_dir, exist_ok=True)

# Raw Financial Data Acquisition

In [14]:
with_basics = True
if index_name == "dow":
    index = DOW_30
elif index_name == "euro":
    index = EURO_STOXX_50
elif index_name == "crypto":
    index = CRYPTO
    with_basics = False
else:
    raise NotImplementedError

for symbol in tqdm(index):
    print(f"Processing {symbol}")
    prepare_data_for_symbol(symbol, data_dir, start_date, end_date, with_basics)

  0%|          | 0/2 [00:00<?, ?it/s]

Processing DOGE-USD


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Processing TRX-USD


100%|██████████| 2/2 [01:41<00:00, 50.91s/it]


# Generate Prompts from Financial Data

In [16]:
# prompts = get_all_prompts("AAPL", 1, 3)
# prompts = get_all_prompts("MSFT", 1, 3, False)
prompts = get_all_prompts("BTC-USD", data_dir, start_date, end_date, min_past_weeks, max_past_weeks, with_basics)

print(prompts[0])

KeyError: 'name'

In [17]:
from prompt_crypto import get_all_prompts
prompts = get_all_prompts("BTC-USD", data_dir, start_date, end_date, min_past_weeks, max_past_weeks, with_basics)
print(prompts[0])

KeyError: 'name'

# Request to GPT-4 for Financial Analysis

In [12]:
# query_gpt4(DOW_30, 1, 3)
query_gpt4(index, data_dir, start_date, end_date, min_past_weeks, max_past_weeks)
# query_gpt4(['WBA'], 1, 4)

WBA - 0
WBA - 1
WBA - 2
WBA - 3
WBA - 4
WBA - 5
WBA - 6
WBA - 7
WBA - 8
WBA - 9
WBA - 10
WBA - 11
WBA - 12
WBA - 13
WBA - 14
WBA - 15
WBA - 16
WBA - 17
WBA - 18
WBA - 19
WMT - 0
WMT - 1
WMT - 2
WMT - 3
WMT - 4
WMT - 5
WMT - 6
WMT - 7
WMT - 8
WMT - 9
WMT - 10
WMT - 11
WMT - 12
WMT - 13
WMT - 14
WMT - 15
WMT - 16
WMT - 17
WMT - 18
WMT - 19
DIS - 0
DIS - 1
DIS - 2
DIS - 3
DIS - 4
DIS - 5
DIS - 6
DIS - 7


# Transform into Llama2 Training Format

In [34]:
# v1
# dow30_dataset = create_dataset(DOW30, True)
# v2
# dow30_nobasic_dataset = create_dataset(DOW_30, 0.8, False)
# v3
dow30_v3_dataset = create_dataset(DOW_30, 0.9)

In [35]:
# dow30_dataset.save_to_disk('fingpt-forecaster-dow30-20230601-20230930-llama')
# dow30_nobasics_dataset.save_to_disk('fingpt-forecaster-dow30nobasics-20230601-20230930-llama')
dow30_v3_dataset.save_to_disk(
    f"fingpt-forecaster-{index_name.lower()}-{start_date.replace('-', '')}-{end_date.replace('-', '')}-{min_past_weeks}-{max_past_weeks}-{str(train_ratio).replace('.', '')}"
)

Saving the dataset (0/1 shards):   0%|          | 0/524 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 58/58 [00:00<00:00, 11222.48 examples/s]


In [36]:
dow30_v3_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'answer', 'period', 'label', 'symbol'],
        num_rows: 524
    })
    test: Dataset({
        features: ['prompt', 'answer', 'period', 'label', 'symbol'],
        num_rows: 58
    })
})

# Test-time Information Fetching

In [23]:
ticker = "AAPL"
n_weeks = 3
curday = get_curday()
data = fetch_all_data(ticker, curday, n_weeks)
data['Basics'] = [json.dumps({})] * len(data)
# data = get_basics(ticker, data, always=True)


[*********************100%%**********************]  1 of 1 completed
/data5/aamayuelasfernandez/FinGPT/fingpt/FinGPT_Forecaster/data_infererence_fetch.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prices.append(stock_data['Close'][i])
/data5/aamayuelasfernandez/FinGPT/fingpt/FinGPT_Forecaster/data_infererence_fetch.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prices.append(stock_data['Close'][-1])


In [24]:
data

,Start Date,End Date,Start Price,End Price,News,Basics
0,2023-12-21,2023-12-28,194.679993,193.580002,"[{""date"": ""20231225193300"", ""headline"": ""What ...",{}
1,2023-12-28,2024-01-04,193.580002,181.910004,"[{""date"": ""20240102022608"", ""headline"": ""Wall ...",{}
2,2024-01-04,2024-01-10,181.910004,186.190002,"[{""date"": ""20240105093748"", ""headline"": ""Micro...",{}


In [25]:
info, prompt = get_all_prompts_online(ticker, data, curday, True)
print(prompt)

[Company Introduction]:

Apple Inc is a leading entity in the Technology sector. Incorporated and publicly traded since 1980-12-12, the company has established its reputation as one of the key players in the market. As of today, Apple Inc has a market capitalization of 2886435.27 in USD, with 15552.75 shares outstanding.

Apple Inc operates primarily in the US, trading under the ticker AAPL on the NASDAQ NMS - GLOBAL MARKET. As a dominant force in the Technology space, the company continues to innovate and drive progress within the industry.

From 2023-12-21 to 2023-12-28, AAPL's stock price decreased from 194.68 to 193.58. Company news during this period are listed below:

[Headline]: 2 models of Apple Watch can go on sale again, for now, after court lifts halt over a patent dispute
[Summary]: Two higher end models of the Apple Watch can go on sale again after a federal court temporarily lifted a sales halt ordered by the International Trade Commission over a patent dispute. The ITC,.